In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

26/01/26 08:27:37 WARN Utils: Your hostname, codespaces-f3c854 resolves to a loopback address: 127.0.0.1; using 10.0.0.133 instead (on interface eth0)
26/01/26 08:27:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


26/01/26 08:27:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

/workspaces/data-engineering-zoomcamp-homework/05-batch/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [6]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [7]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [8]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [9]:
df_green_revenue.show()

[Stage 14:===================>                                      (1 + 2) / 3]

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-28 19:00:00| 134|193.61000000000007|            17|
|2020-01-22 19:00:00|  65| 657.0300000000001|            41|
|2020-01-27 08:00:00|  17|             85.56|             4|
|2020-01-02 09:00:00|  66|229.39999999999998|            12|
|2020-01-02 12:00:00|  89|310.28000000000003|            14|
|2020-01-07 12:00:00|  66|             179.5|             9|
|2020-01-03 08:00:00| 223|165.90000000000003|             9|
|2020-01-17 10:00:00|  41| 638.2699999999999|            49|
|2020-01-02 11:00:00|  26|             198.6|             7|
|2020-01-12 20:00:00| 247|36.900000000000006|             3|
|2020-01-12 15:00:00|  10|             75.92|             1|
|2020-01-24 04:00:00| 129|148.89000000000001|            13|
|2020-01-04 20:00:00|  25| 369.5700000000001|            23|
|2020-01-22 10:00:00|  7

In [10]:
df_yellow_revenue.show()

[Stage 17:===================================================>    (11 + 1) / 12]

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-10 06:00:00| 144|            149.51|             9|
|2020-01-29 03:00:00|  13|            248.89|             6|
|2020-01-13 21:00:00| 255|            108.98|             8|
|2020-01-04 12:00:00| 249| 2245.569999999999|           152|
|2020-01-02 15:00:00| 236| 7382.910000000018|           504|
|2020-01-31 21:00:00|  50|1784.4399999999994|           104|
|2020-01-07 12:00:00|  68| 3046.229999999997|           181|
|2020-01-02 09:00:00| 161|4299.9299999999985|           242|
|2020-01-23 17:00:00| 233| 4642.769999999999|           235|
|2020-01-06 22:00:00| 158| 923.2100000000003|            57|
|2020-01-15 07:00:00|  13|2432.6299999999997|           118|
|2020-01-23 19:00:00| 224|            327.28|            16|
|2020-01-18 00:00:00| 144|2951.2999999999984|           174|
|2020-01-17 19:00:00| 17

In [11]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [12]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [13]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [14]:
df_join.show()

[Stage 26:>                                                         (0 + 1) / 1]

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  13|              null|                null|1214.8000000000002|                   56|
|2020-01-01 00:00:00|  18|               7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:00|  29|  

In [15]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [16]:
df_join = spark.read.parquet('data/report/revenue/total')

In [17]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [18]:
df_zones = spark.read.parquet('zones/')

In [19]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [20]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|         3|    Bronx|Allerton/Pelham G...|   Boro Zone|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|         4|Manhattan|       Alphabet City| Yellow Zone|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000001|                   38|         7|   Queens|             Astoria|   Boro Zone|
|2020-01-01 00:00:00| 

In [21]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')